# Alternate view 06

### Purpose
- Demonstrate graph connecting output to input. The example shows connection between cities in a trip. 
- Aggregate view at the country levels. The graph only shows flights between countries.

## Imports modules and does some configurations

In [56]:
import logging
logging.basicConfig()
logger = logging.getLogger('dependencynet')
logger.setLevel(logging.WARN)

In [57]:
# clarification for linter
from IPython.display import display

In [58]:
# remove these lines if you use the pypi package
import sys
sys.path.append("../../..") # go to parent dir

In [59]:
from dependencynet.schema import SchemaBuilder
from dependencynet.model import ModelBuilder
from dependencynet.network.graphbuilder import GraphBuilder, LevelNode, InputNode, OutputNode
from dependencynet.network.stylebuilder import StyleBuilder
from dependencynet.network.graphviewer import GraphViewer
from dependencynet.network.graphml import GraphMLConverter

## Loads and prepare data

Check the notebook for more information on the dataset and the steps requiored to prepare the graph.

In [60]:
%run example-graphml-trips.ipynb

,area,country,town,flight_in,flight_out
0,Europe,France,Paris,fl2,fl3
1,Europe,France,Lyon,NaN,fl1
2,Europe,UK,London,fl4,fl5
3,Europe,Italia,Rome,fl1,fl2
4,Asia,Japan,Tokyo,fl3,fl4


CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeSpacing': 10, 'edgeLenghVal': 10, 'rankDir': 'LR'}, cy…

Done


In [61]:
graph_model.pretty_print()

['Nodes:',
 'area level - A01 Europe',
 'area level - A02 Asia',
 'country level - A01C01 France',
 'country level - A01C02 UK',
 'country level - A01C03 Italia',
 'country level - A02C01 Japan',
 'town level - A01C01T01 Paris',
 'town level - A01C01T02 Lyon',
 'town level - A01C02T01 London',
 'town level - A01C03T01 Rome',
 'town level - A02C01T01 Tokyo',
 'flight_in input resource - A01C01T01FIn01 fl2',
 'flight_in input resource - A01C02T01FIn01 fl4',
 'flight_in input resource - A01C03T01FIn01 fl1',
 'flight_in input resource - A02C01T01FIn01 fl3',
 'flight_out output resource - A01C01T01FOut01 fl3',
 'flight_out output resource - A01C01T02FOut01 fl1',
 'flight_out output resource - A01C02T01FOut01 fl5',
 'flight_out output resource - A01C03T01FOut01 fl2',
 'flight_out output resource - A02C01T01FOut01 fl4',
 '',
 'Edges:',
 'area level A01 -> country level A01C01',
 'area level A01 -> country level A01C02',
 'area level A01 -> country level A01C03',
 'area level A02 -> country le

## Alter the graph model 

- remove town nodes and aggtegate by country
- connect flight_in and flight_out into a single node

In [62]:
graph_model.aggregate_level(['area', 'country'])
graph_model.merge_connection('flight_out', 'flight_in', 'flight')

In [63]:
graph_model.pretty_print()

['Nodes:',
 'area level - A01 Europe',
 'area level - A02 Asia',
 'country level - A01C01 France',
 'country level - A01C02 UK',
 'country level - A01C03 Italia',
 'country level - A02C01 Japan',
 'flight resource - fl3',
 'flight resource - fl1',
 'flight resource - fl2',
 'flight resource - fl4',
 '',
 'Edges:',
 'area level A01 -> country level A01C01',
 'area level A01 -> country level A01C02',
 'area level A01 -> country level A01C03',
 'area level A02 -> country level A02C01',
 'country level A01C01 -> flight resource fl3',
 'country level A01C01 -> flight resource fl1',
 'country level A01C03 -> flight resource fl2',
 'country level A02C01 -> flight resource fl4',
 'flight resource fl3 -> country level A02C01',
 'flight resource fl1 -> country level A01C03',
 'flight resource fl2 -> country level A01C01',
 'flight resource fl4 -> country level A01C02']

#### Shows graph

swith layout to klay for a more compact graph

In [64]:
display(GraphViewer(graph_model).render('klay', graph_style, 'LR'))

CytoscapeWidget(cytoscape_layout={'name': 'klay', 'nodeSpacing': 10, 'edgeLenghVal': 10, 'rankDir': 'LR'}, cyt…

## Rework the graph model 

remove area and flights, show only connections between countries

In [65]:
graph_model_alt = GraphBuilder().with_types(class_mapping).with_model(model).render()
graph_model_alt.aggregate_level(['area', 'country'])
graph_model_alt.merge_connection('flight_out', 'flight_in', 'flight')
graph_model_alt.remove_category('area')
graph_model_alt.fold_category('flight')

#### Shows graph

In [66]:
display(GraphViewer(graph_model_alt).render('klay', graph_style, 'LR'))

CytoscapeWidget(cytoscape_layout={'name': 'klay', 'nodeSpacing': 10, 'edgeLenghVal': 10, 'rankDir': 'LR'}, cyt…

## Exports to GraphML

In [67]:
dirname = path.join('output')
makedirs(dirname, exist_ok=True)
filename = path.join(dirname, 'trips_yed_06.graphml')
GraphMLConverter(graph_model_alt, graph_style, schema_trips).save(filename)